In [1]:
# Python Dependencies
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date,when
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Store filepath in a variable
constituents_path = Path("./Resources/constituents_sh.csv")

In [5]:
# 1. Read CSV file 
constituents_df = spark.read.csv(str(constituents_path), sep=",", header=True)
# Show the dataframe
constituents_df.show()

+--------------+-----------+-----+----------+-------------+----------------+--------------------+---------------+-----------------+---------------+---------------+--------------------+------------------+----------------+----------------+--------------------+--------------------+------------------+------------------+--------------------+----------+----+-----------+--------------------+--------------------+------+--------------+--------------------+---------------------+----+-----------------------+--------------------+---------------------+-----------------+
|Constituent ID|       City|State|  Postcode|      Country|Constituent type|   Constituent codes|Lifetime giving|First gift amount|First gift type|First gift date|     First gift fund|Latest gift amount|Latest gift type|Latest gift date|    Latest gift fund|Greatest gift amount|Greatest gift type|Greatest gift date|  Greatest gift fund| Birthdate| Age|AgeInMonths|   PrimaryEmployment|           Education|Gender|Marital status|      

In [6]:
constituents_df.columns

['Constituent ID',
 'City',
 'State',
 'Postcode',
 'Country',
 'Constituent type',
 'Constituent codes',
 'Lifetime giving',
 'First gift amount',
 'First gift type',
 'First gift date',
 'First gift fund',
 'Latest gift amount',
 'Latest gift type',
 'Latest gift date',
 'Latest gift fund',
 'Greatest gift amount',
 'Greatest gift type',
 'Greatest gift date',
 'Greatest gift fund',
 'Birthdate',
 'Age',
 'AgeInMonths',
 'PrimaryEmployment',
 'Education',
 'Gender',
 'Marital status',
 'Solicit codes',
 'Student Organizations',
 'Arts',
 'Fraternal Organizations',
 "Men's Athletics",
 'Scholarship Recipient',
 "Women's Athletics"]

In [7]:
constituents_df.printSchema()

root
 |-- Constituent ID: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Constituent type: string (nullable = true)
 |-- Constituent codes: string (nullable = true)
 |-- Lifetime giving: string (nullable = true)
 |-- First gift amount: string (nullable = true)
 |-- First gift type: string (nullable = true)
 |-- First gift date: string (nullable = true)
 |-- First gift fund: string (nullable = true)
 |-- Latest gift amount: string (nullable = true)
 |-- Latest gift type: string (nullable = true)
 |-- Latest gift date: string (nullable = true)
 |-- Latest gift fund: string (nullable = true)
 |-- Greatest gift amount: string (nullable = true)
 |-- Greatest gift type: string (nullable = true)
 |-- Greatest gift date: string (nullable = true)
 |-- Greatest gift fund: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Age: string (nullable

In [8]:
#Change amounts coulumns to decimals 
constituents_df = constituents_df.withColumn("Lifetime giving", col("Lifetime giving").cast("decimal"))
constituents_df = constituents_df.withColumn("First gift amount", col("First gift amount").cast("decimal"))
onstituents_df = constituents_df.withColumn("Latest gift amount", col("Latest gift amount").cast("decimal"))
constituents_df = constituents_df.withColumn("Greatest gift amount", col("Greatest gift amount").cast("decimal"))
# Change to date format
#constituents_df = constituents_df.withColumn("First gift date", to_date(col("First gift date"), "MM-dd-yyyy"))
#constituents_df = constituents_df.withColumn("Latest gift date", to_date(col("Latest gift date"), "MM-dd-yyyy"))
#constituents_df = constituents_df.withColumn("Greatest gift date", to_date(col("Greatest gift date"), "MM-dd-yyyy"))

In [9]:
#Add new column for repeated 
constituents_df = constituents_df.withColumn('repeatedConstituents', when(col('Lifetime giving') > col('First gift amount'), 'Yes').otherwise('No'))

In [10]:
#Create a temporary view of the DataFrame.
constituents_df.createOrReplaceTempView('constituents')

In [16]:
# Check the view data
spark.sql("""SELECT 
            `Constituent ID`,`Lifetime giving`,
             `First gift amount`,`First gift date`,
             `Latest gift amount`,`Latest gift date`,
             `Greatest gift amount`,`Greatest gift date`,
             repeatedConstituents
             
   FROM 
       constituents 
   WHERE 
   repeatedConstituents='Yes'
    
   """).show()

+--------------+---------------+-----------------+---------------+------------------+----------------+--------------------+------------------+--------------------+
|Constituent ID|Lifetime giving|First gift amount|First gift date|Latest gift amount|Latest gift date|Greatest gift amount|Greatest gift date|repeatedConstituents|
+--------------+---------------+-----------------+---------------+------------------+----------------+--------------------+------------------+--------------------+
|         36583|            450|               50|      4/12/2005|            200.00|      12/29/2023|                 200|        12/29/2023|                 Yes|
|         21368|            600|              100|       1/1/1970|            500.00|      12/19/1985|                 500|        12/19/1985|                 Yes|
|         44089|            350|              300|     10/25/2022|            250.00|       4/10/2024|                 300|        10/25/2022|                 Yes|
|         39798|

In [12]:
#Count distinct donors 
spark.sql("""SELECT 
            count(distinct(`Constituent ID`))
   FROM 
       constituents 
    
   """).show()

+------------------------------+
|count(DISTINCT Constituent ID)|
+------------------------------+
|                         50913|
+------------------------------+



In [13]:
#count the number of donors whos has not contribute yet 
spark.sql("""SELECT 
            count(distinct(`Constituent ID`))
   FROM 
       constituents 
   WHERE 
    `First gift amount`=0
    
   """).show()

+------------------------------+
|count(DISTINCT Constituent ID)|
+------------------------------+
|                         39140|
+------------------------------+



In [18]:
#Repeated donors
spark.sql("""SELECT 
        count(*)
   FROM 
       constituents 
   WHERE 
   `repeatedConstituents`='Yes'
   """).show()


+--------+
|count(1)|
+--------+
|    4358|
+--------+



In [32]:
#Null information among the active donors
spark.sql("""
   SELECT 
      COUNT(`Constituent ID`)
   FROM 
       constituents 
   WHERE 
      `First gift amount` > 0 AND City IS NULL

""").show()

+---------------------+
|count(Constituent ID)|
+---------------------+
|                  150|
+---------------------+



In [30]:
#Group records as per the funds 
spark.sql("""
   SELECT 
      `First gift fund`,count(`Constituent ID`) as totConstituent
   FROM 
       constituents 
   WHERE 
      `First gift amount` > 0 
  GROUP BY  `First gift fund`

""").show()

+--------------------+--------------+
|     First gift fund|totConstituent|
+--------------------+--------------+
|Women's Basketbal...|           310|
|Business and Econ...|            34|
|Multicultural Cen...|            11|
|Mary Jo Gangnon M...|             9|
|General Charles C...|             8|
|0-Francis C DeVin...|             2|
|Davis, John Bloom...|            46|
|Robert Banks Memo...|            11|
|     Mock Trial Fund|            37|
|Superior Voices S...|             5|
| Women's Tennis Fund|            65|
|Everson, Dr. and ...|             7|
|Boswell, Melinda,...|             1|
|0-Center Court Cl...|            14|
|Kyllo, Hester, Sc...|             1|
|Distance Learning...|             9|
| Women's Hockey Fund|           146|
|Grad Psych Develo...|             1|
|SCCU Stadium Dan ...|            13|
|Brown, Maurice, S...|             4|
+--------------------+--------------+
only showing top 20 rows



In [15]:
#Convert to Pandas dataframe
constituents_pd_df = constituents_df.toPandas()
constituents_pd_df.head()

,Constituent ID,City,State,Postcode,Country,Constituent type,Constituent codes,Lifetime giving,First gift amount,First gift type,...,Gender,Marital status,Solicit codes,Student Organizations,Arts,Fraternal Organizations,Men's Athletics,Scholarship Recipient,Women's Athletics,repeatedConstituents
0,60472,Superior,WI,54880-2556,United States,Organization,Business (No start date - No end date),0,0,None,...,None,None,None,None,None,None,None,None,None,No
1,43735,Hermantown,MN,55811-1755,United States,Organization,Other Organizations (8/29/2012 - No end date),50,50,One-time gift,...,None,None,None,None,None,None,None,None,None,No
2,60145,Esko,MN,55733-9645,United States,Organization,Other Organizations (No start date - No end date),50,50,One-time gift,...,None,None,None,None,None,None,None,None,None,No
3,21332,None,None,None,United States,Organization,Business (No start date - No end date),25,25,One-time gift,...,None,None,None,None,None,None,None,None,None,No
4,50410,Superior,WI,54880-1504,United States,Organization,Business (No start date - No end date),0,0,None,...,None,None,None,None,None,None,None,None,None,No
